# Working with Structure Function in lsstseries

In [ ]:
import numpy as np

from lsstseries.analysis.structurefunction2 import calc_sf2

from lsstseries.analysis.structure_function.base_argument_container import StructureFunctionArgumentContainer
from lsstseries.analysis.structure_function.basic.calculator import BasicStructureFunctionCalculator

import lsstseries

# make sure we're running the code we think we're running
print(lsstseries.__file__)

Over time there will be several different Structure Function calculation methods implemented.
Each of the calculation method will be a child class of the StructureFunctionCalculator base class.

A dictionary of all available SF calculation methods dynamically generated at run time and is available as the SF_METHODS dictionary.
The dictionary is a mapping of unique SF calculation method name to calculator class. 
Generally a user would not interact with `SF_METHODS` other than to view the list of SF calculation method names.

In [ ]:
from lsstseries.analysis.structure_function import SF_METHODS
print(SF_METHODS.keys())

Each Structure Function calculation instance requires an input argument container object.
But for those who are a class method, `expected_argument_container` is provided that will return the expected argument container class.

Most of the time the user will not have to be aware of this functionality because default container will be created when needed. 
In the interest of completeness, we show below how to check for the expected argument container class, instantiate one, and check
the value of one of the arguments it contains.

In [ ]:
# Note that we do not need to create an instance of the BasicStructureFunctionCalculator to call the `expected_argument_container` (static)method.
arg_container_type = BasicStructureFunctionCalculator.expected_argument_container()
arg_container = arg_container_type()

# Print out the default value for one particular argument.
print(arg_container.bin_count_target)

Using the argument container created above, we can create an instance of a Structure Function calculator method.

In [ ]:
# Create an instance of a calculator using the dictionary, and call the `calculate` method
basic_calculator = SF_METHODS["basic"]([], [], [], arg_container)

# given no inputs, we expect an empty output
basic_calculator.calculate()

The vast majority of the time, users will not interact directly with the Structure Function calculation method.
It is more likely they will call the `calc_sf2` function either directly or via an `Ensemble` or `Timeseries`.

Below we show various examples of calling `calc_sf2` directly.

In [ ]:
times = [1.11, 2.23, 3.45, 4.01, 5.67, 6.32, 7.88, 8.2]
fluxes = [0.11, 0.23, 0.45, 0.01, 0.67, 0.32, 0.88, 0.2]
errors = [0.1, 0.023, 0.045, 0.1, 0.067, 0.032, 0.8, 0.02]
bands = np.array(["r"] * len(fluxes))
lightcurve_ids = [1, 1, 1, 1, 1, 1, 1, 1]

res = calc_sf2(times, fluxes, errors, bands, lightcurve_ids)

print(res)

In [ ]:
# Same as above, but we explicitly create an argument_container and produce the same results as before

times = [1.11, 2.23, 3.45, 4.01, 5.67, 6.32, 7.88, 8.2]
fluxes = [0.11, 0.23, 0.45, 0.01, 0.67, 0.32, 0.88, 0.2]
errors = [0.1, 0.023, 0.045, 0.1, 0.067, 0.032, 0.8, 0.02]
bands = np.array(["r"] * len(fluxes))
lightcurve_ids = [1, 1, 1, 1, 1, 1, 1, 1]

arg_container = StructureFunctionArgumentContainer()

res = calc_sf2(times, fluxes, errors, bands, lightcurve_ids, argument_container=arg_container)

print(res)

In [ ]:
# Same as before, but being more explicit about the arguments in the argument container

times = [1.11, 2.23, 3.45, 4.01, 5.67, 6.32, 7.88, 8.2]
fluxes = [0.11, 0.23, 0.45, 0.01, 0.67, 0.32, 0.88, 0.2]
errors = [0.1, 0.023, 0.045, 0.1, 0.067, 0.032, 0.8, 0.02]
bands = np.array(["r"] * len(fluxes))
lightcurve_ids = [1, 1, 1, 1, 1, 1, 1, 1]

# Note, not all arguments need to be provided, nor do they have to be set at the
# time the object is instantiated.
arg_container = StructureFunctionArgumentContainer(band_to_calc=None, combine=False)
arg_container.bins = None
arg_container.method = "size"

res = calc_sf2(times, fluxes, errors, bands, lightcurve_ids, argument_container=arg_container)

print(res)

Next we show how one would work with `calc_sf2` using an `Ensemble` object.
For more information about `Ensembles` see the example code in the "Working with the lsstseries Ensemble object" notebook.

In [ ]:
from lsstseries.ensemble import Ensemble

ens = Ensemble()  # initialize an ensemble object

# Read in data from a parquet file
ens.from_parquet("../../tests/lsstseries_tests/data/source/test_source.parquet",
                id_col='ps1_objid',
                time_col='midPointTai',
                flux_col='psFlux',
                err_col='psFluxErr',
                band_col='filterName')

In [ ]:
# Call batch on the ensemble providing no additional arguments.
res = ens.batch(calc_sf2, compute=True)
res

In [ ]:
# Create a StructureFunctionArgumentContainer with lots of non-default argument values

arg_container = StructureFunctionArgumentContainer()
arg_container.band_to_calc = ["r"]
arg_container.combine = True
arg_container.bin_method = "loglength"
arg_container.bin_count_target = 40

res = ens.batch(calc_sf2, compute=True, argument_container=arg_container)
res

In [ ]:
ens.client.close()

Here we show how to work with `calc_sf2` using a `TimeSeries` object. 
For more information about working with `TimeSeries` objects, see the example code in the "Working with the lsstseries Timeseries object" notebook.

In [ ]:
# Pull out a TimeSeries object from the previous Ensemble
ts = ens.to_timeseries(88472935274829959) # provided a target object id

res = ts.sf2()
print(res)

In [ ]:
# Create a StructureFunctionArgumentContainer with lots of non-default argument values

arg_container = StructureFunctionArgumentContainer()
arg_container.band_to_calc = ["r"]
arg_container.combine = True
arg_container.bin_method = "loglength"
arg_container.bin_count_target = 40

res = ts.sf2(argument_container=arg_container)
print(res)

## Implementing a custom Structure Function calculator

There may be times when the provided set of Structure Function calculator methods is insufficient for your work.
It is possible to extend the `StructureFunctionCalculator` base class to create your own calculator method while making use of foundational code.

The example below uses the `calculate` method from the `basic` structure function calculator method, and shows how the data is typically manipulated.
Note that a unique name must be provided in the `name_id` static method.
This is the name that will be provided when calling the `calc_sf2` function to determine which SF calculator method to use.

Note that before this new SF calculator method must be registered before it is used.
We demonstrate how to register the new calculator method at the end of the next cell. 

In [ ]:
# Creating a new subclass of `StructureFunctionCalculator` to implement a new SF calculator method
from lsstseries.analysis.structure_function.base_calculator import StructureFunctionCalculator

class ExperimentalStructureFunctionCalculator(StructureFunctionCalculator):
    def calculate(self):
        for lc_idx in range(len(self._time)):
            lc_times = self._time[lc_idx]
            lc_fluxes = self._flux[lc_idx]
            lc_errors = self._err[lc_idx]

            # mask out any nan values
            t_mask = np.isnan(lc_times)
            f_mask = np.isnan(lc_fluxes)
            e_mask = np.isnan(lc_errors)  # always mask out nan errors?
            lc_mask = np.logical_or(t_mask, f_mask, e_mask)

            lc_times = lc_times[~lc_mask]
            lc_fluxes = lc_fluxes[~lc_mask]
            lc_errors = lc_errors[~lc_mask]

            # d_times - difference of times
            dt_matrix = lc_times.reshape((1, lc_times.size)) - lc_times.reshape((lc_times.size, 1))
            d_times = dt_matrix[dt_matrix > 0].flatten()

            # d_fluxes - difference of fluxes
            df_matrix = lc_fluxes.reshape((1, lc_fluxes.size)) - lc_fluxes.reshape((lc_fluxes.size, 1))
            d_fluxes = df_matrix[dt_matrix > 0].flatten()

            # err^2 - errors squared
            err2_matrix = (
                lc_errors.reshape((1, lc_errors.size)) ** 2 + lc_errors.reshape((lc_errors.size, 1)) ** 2
            )
            err2s = err2_matrix[dt_matrix > 0].flatten()

            # corrected each pair of observations
            cor_flux2 = d_fluxes**2 - err2s

            # build stack of times and fluxes
            self._dts.append(d_times)
            self._all_d_fluxes.append(cor_flux2)

        # self._calculate_binned_statistic is provided by the parent class and
        # will operate on self._dts and self._all_d_fluxes by default.
        dts, sfs = self._calculate_binned_statistics()

        return dts, sfs

    @staticmethod
    def name_id() -> str:
        return "experimental"

    @staticmethod
    def expected_argument_container() -> type:
        return StructureFunctionArgumentContainer


# Registering the new calculator method using `register_subclasses`
from lsstseries.analysis.structure_function import SF_METHODS
from lsstseries.analysis.structure_function.base_calculator import register_sf_subclasses

print("Current list of Structure Function calculator methods")
print(SF_METHODS.keys())

# register the new subclass
SF_METHODS = register_sf_subclasses()

print("Updated list now includes the new 'experimental' method.")
print(SF_METHODS.keys())

We can now make use of the updated registry of Structure Function calculator methods as shown below.

In [ ]:
# Create an instance of our new calculator method using the dictionary, and call the `calculate` method
experimental_calculator = SF_METHODS["experimental"]([], [], [], arg_container)

# given no inputs, we expect an empty output
experimental_calculator.calculate()